This notebook will process the music files previously created and build the melody descriptors using Melodia.

There are three processes here. The first one will take the original music files, the second one will process the  vocals-only, and the third one will take the accompaniments.

A final process will take the descriptors created by the last two and concatenate them giving the vocals priority over the accompaniments. This last process was created for future experiments where the full song is present but the vocals, in theory, will be described in a more clear way given that there is no accompaniment sounds. This would help in cases where the user hums both the accompaniment and the lyrics of a song.

In [1]:
import os
import essentia.standard
import numpy as np

In [2]:
def read_melody_files(filename):
    with open(filename) as f:
        _notes = f.read()

    _notes = _notes.split(',')
    return [float(i) for i in _notes]

In [ ]:
path = 'dataset/sources/'
vocals = os.listdir(path)

In [ ]:
for song in vocals:
    loader = essentia.standard.EqloudLoader(filename=path + song, sampleRate=44100)
    audio = loader()
    pitch_extractor = essentia.standard.PredominantPitchMelodia(frameSize=2048, hopSize=180)
    pitch_values, _ = pitch_extractor(audio)

    with open('dataset/melodies/full_songs/' + song.split('.')[0] + '.txt', 'w') as writer:
        writer.write(','.join(str(e) for e in pitch_values))

In [3]:
path = 'dataset/results/vocals_spleeter/'
#path = 'dataset/results/vocals_demucs/'
vocals = os.listdir(path)

In [4]:
for vocal in vocals:
    loader = essentia.standard.EqloudLoader(filename=path + vocal, sampleRate=44100)
    audio = loader()
    pitch_extractor = essentia.standard.PredominantPitchMelodia(frameSize=2048, hopSize=180)
    pitch_values, _ = pitch_extractor(audio)

    with open('dataset/melodies/vocals_spleeter/' + vocal.split('.')[0] + '.txt', 'w') as writer:
        writer.write(','.join(str(e) for e in pitch_values))

In [ ]:
path = 'dataset/results/accompaniments/'
accompaniments = os.listdir(path)

In [ ]:
for accompaniment in accompaniments:
    loader = essentia.standard.EqloudLoader(filename=path + accompaniment, sampleRate=44100)
    audio = loader()
    pitch_extractor = essentia.standard.PredominantPitchMelodia(frameSize=2048, hopSize=50)
    pitch_values, _ = pitch_extractor(audio)

    with open('dataset/melodies/accompaniments/' + accompaniment.split('.')[0] + '.txt', 'w') as writer:
        writer.write(','.join(str(e) for e in pitch_values))

In [ ]:
vocals_path = 'dataset/melodies/vocals_spleeter/'
#vocals_path = 'dataset/melodies/vocals_demucs/'
vocals = os.listdir(vocals_path)
accompaniments_path = 'dataset/melodies/accompaniments/'
accompaniments = os.listdir(accompaniments_path)
results_path = 'dataset/melodies/concatenations/'

In [ ]:
for vocal in vocals:
    for accompaniment in accompaniments:
        if vocal == accompaniment:
            vocal_string = read_melody_files(vocals_path + vocal)
            accompaniment_string = read_melody_files(accompaniments_path + accompaniment)

            result = np.zeros(len(vocal_string))
            
            for i in range(len(vocal_string)):
                if vocal_string[i] != 0:
                    result[i] = vocal_string[i]
                else:
                    if accompaniment_string[i] != 0:
                        result[i] = accompaniment_string[i]
            
            with open(results_path + vocal, 'w') as writer:
                writer.write(','.join(str(e) for e in result))